In [10]:
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import services.influencers_sevice as influencers_service
#visualisation 
import pyLDAvis
import pyLDAvis.gensim
import gensim
import gensim.corpora as corpora
import json 
import glob
import nltk
from nltk.corpus import stopwords

In [11]:
posts=influencers_service.get_posts()

In [12]:
def merge_content_posts(posts) : 
    #concatenation of captions of all posts
    merged_content = ""
    for post in posts:
         #extract caption text
         caption = post.get('caption')
         if (caption):
            merged_content += caption + " "
    return merged_content

In [13]:
def gen_words(text):
    words=[]
    new = gensim.utils.simple_preprocess(text, deacc=True)    
    words.append(new)
    return words

In [14]:
def gen_bow(words):
    id2word = corpora.Dictionary(words)
    corpus=[]
    for word in words: 
        new = id2word.doc2bow(word)
        corpus.append(new)
    return  corpus

In [15]:
def make_bigrams(texts,bigram):
    return [bigram[doc] for doc in texts]

def make_trigrams(texts,bigram,trigram):
    return [trigram[bigram[doc]] for doc in texts]

In [16]:
# posts=posts[0:3] #for rapid testing we only use three posts
merge=merge_content_posts(posts)
# print(merge)
words=gen_words(merge)
# print(words)

##generate bigrams and tigrams##
bigram_phrases = gensim.models.Phrases(words, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[words], threshold=100)
bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)
data_bigrams = make_bigrams(words,bigram)
data_bigrams_trigrams = make_trigrams(data_bigrams,bigram,trigram)


In [17]:
from gensim.parsing.preprocessing import STOPWORDS
texts = data_bigrams_trigrams #for easier typing
custom_stopwords = {'god','bless','tunisia','la','ly','et','al','fy','mn','mei'}  # Add any additional stop words to this set
stopwords = STOPWORDS.union(custom_stopwords)
filtered_texts = []
for text in texts:
    filtered_text = [word for word in text if word not in stopwords]
    filtered_texts.append(filtered_text)
texts=filtered_texts
# print(texts)

In [18]:
# # #TF-IDF REMOVAL
from gensim.models import TfidfModel
# 
id2word = corpora.Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]
tfidf = TfidfModel(corpus, id2word=id2word,smartirs='ntc')

low_value =0.0005
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing
    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

# print(corpus)

In [19]:
lda_model=gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=5,
    random_state=50,
    chunksize=1000,
    update_every=1,
    passes=1,
    alpha=0.01,
    eta=0.002,
)

In [20]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30,lambda_step=0.01)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.002837 -0.002553       1        1  69.426353
2     -0.000321  0.003467       2        1  29.518110
1     -0.002374 -0.004139       3        1   0.641211
4     -0.004086  0.001379       4        1   0.414306
3      0.003944  0.001847       5        1   0.000021, topic_info=                  Term        Freq       Total Category  logprob  loglift
5474              love  722.000000  722.000000  Default  30.0000  30.0000
4041             happy  258.000000  258.000000  Default  29.0000  29.0000
2598             dress  224.000000  224.000000  Default  28.0000  28.0000
2222               day  185.000000  185.000000  Default  27.0000  27.0000
10349            thank  246.000000  246.000000  Default  26.0000  26.0000
...                ...         ...         ...      ...      ...      ...
10345  tflers_tweegram    0.000030  173.211242   Topic5  -5.8189  -0.1992
6541              memy    0.000023  107.638048   Topic5  -6.0945   0.0009
3785              good    0.000026  144.667123   Topic5  -5.9694  -0.1697
261          alikaroui    0.000024  119.860017   Topic5  -6.0622  -0.0744
820          beautiful    0.000024  164.362920   Topic5  -6.0442  -0.3720

[456 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
47         1  0.635846  acceptance
47         2  0.635846  acceptance
85         1  0.698315     actress
85         2  0.304394     actress
244        1  1.276258         alf
...      ...       ...         ...
11382      1  0.749396     youtube
11382      2  0.244185     youtube
11382      3  0.008420     youtube
11382      4  0.008420     youtube
11459      1  0.855299          zo

[526 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 5, 4])